<a href="https://colab.research.google.com/github/ManikaNagpal/PPro/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_root='/content/drive/My Drive/ChatBot' #Please upload the files in your drive and change the path to it accordingly.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import string
import random 

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [9]:
data_file = open(data_root + '/intents.json').read()
data = json.loads(data_file)

data

{'intents': [{'context': '',
   'patterns': ['Hello', 'Hi there', 'Good morning', "What's up"],
   'responses': ['Hey!', 'Hello', 'Hi!', 'Good morning!'],
   'tag': 'hello'},
  {'context': [''],
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'tag': 'noanswer'},
  {'context': '',
   'patterns': ['What is your job', 'What is your work'],
   'responses': ['My job is to make you feel like everything is okay.',
    'I work to serve you as well as possible'],
   'tag': 'job'},
  {'context': '',
   'patterns': ['What is your age', 'How old are you', 'When were you born'],
   'responses': ['I was born in 2021'],
   'tag': 'age'},
  {'context': '',
   'patterns': ['How are you today', 'How are you'],
   'responses': ['I am feeling good, you?',
    'Very good and you?',
    "Actually, I'm okay and you?"],
   'tag': 'feeling'},
  {'context': 'feeling',
   'patterns': ['I am good too',
    'I feel fine',
    'Good

In [10]:
# Each list to create
words = []
classes = []
data_X = []
data_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_X.append(pattern)
        data_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

['Hello', 'Hi', 'there', 'Good', 'morning', 'What', "'s", 'up', 'What', 'is', 'your', 'job', 'What', 'is', 'your', 'work', 'What', 'is', 'your', 'age', 'How', 'old', 'are', 'you', 'When', 'were', 'you', 'born', 'How', 'are', 'you', 'today', 'How', 'are', 'you', 'I', 'am', 'good', 'too', 'I', 'feel', 'fine', 'Good', '!', 'Fine', 'I', 'am', 'good', 'I', 'am', 'great', 'great', 'I', 'am', 'feeling', 'bad', 'No', 'I', 'am', 'sad', 'No', 'What', 'can', 'you', 'do', 'What', 'can', 'I', 'ask', 'you', 'Can', 'you', 'help', 'me', 'Are', 'you', 'a', 'girl', 'You', 'are', 'a', 'women', 'Are', 'you', 'a', 'men', 'Are', 'you', 'a', 'boy', 'Thank', 'you', 'Thank', 'you', 'very', 'much', 'thanks', 'Goodbye', 'Good', 'afternoon', 'Bye', 'Where', 'do', 'you', 'live', 'What', 'are', 'you', 'doing', 'Can', 'you', 'wait', '2', 'minutes', 'Please', 'wait', 'Wait', '2', 'secs', 'please', 'Are', 'you', 'still', 'there', '?', 'Are', 'you', 'here', '?']


In [ ]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(data_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

shape(train_Y)

In [ ]:
# The Deep Learning Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=180, verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               7552      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                1040      
Total params: 16,848
Trainable params: 16,848
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
2/2 [==============================] - 0s 9ms/step - loss: 2.7744 - accuracy: 0.0732
Epoch 2/200
2/

In [ ]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0] #Extracting probabilities
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True) #Sorting by values of probability in decreasing order
  print("y_pred:",y_pred)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]]) #Contains labels(tags) for highest probability
  print(return_list)  
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
# running the chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

Hi
So, everything's okay!
Hello
That is perfect!
What do you do?
I can do a lot of things but here are some of my skills, you can ask me: the capital of a country, its currency and its area. A random number. To calculate a math operation.
I see
So, everything's okay!
Yes
So, everything's okay!
Oh No
That is perfect!
No it is not
I was born in 2021
Bye
So, everything's okay!


KeyboardInterrupt: ignored